Here we present a simple example of how to use fine-tuning with the hugging face setfit library.

Make sure you run `pip install setfit` before running the code below.

If you want to use a different library, you can use the hugging face transformers library, which is more comprehensive and flexible but also more complex and not designed for compute-restricted environments.

In [1]:
import numpy as np
import random
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sklearn.metrics import accuracy_score

In [2]:
# Base sentiment examples
positive_reviews = [
    "I love this product, it's amazing!",
    "Great experience, highly recommend",
    "Absolutely fantastic service",
    "Best purchase I've ever made",
    "The quality exceeded my expectations",
    "Customer service was outstanding",
    "Works perfectly for my needs",
    "Really happy with this purchase",
    "Great value for money",
    "Would definitely buy again"
]

negative_reviews = [
    "This is terrible, don't buy it",
    "Waste of money, very disappointed",
    "Poor quality and bad customer service",
    "Completely failed to meet expectations",
    "Would not recommend to anyone",
    "Save your money and avoid this",
    "Terrible experience overall",
    "Product broke after first use",
    "Customer support was unhelpful",
    "Not worth the price at all"
]

def add_noise_to_text(text):
    noise_words = [
        "today", "yesterday", "recently", "definitely", "absolutely",
        "quite", "very", "extremely", "somewhat", "rather"
    ]
    words = text.split()
    # 50% chance to add a noise word at the start
    if random.random() > 0.5:
        words.insert(0, random.choice(noise_words))
    # 50% chance to add a noise word at the end
    if random.random() > 0.5:
        words.append(random.choice(noise_words))
    return " ".join(words)

# Generate balanced dataset with variations
texts = []
labels = []
num_examples = 50  # 50 examples of each sentiment

for _ in range(num_examples):
    # Add positive example
    texts.append(add_noise_to_text(random.choice(positive_reviews)))
    labels.append(1)
    
    # Add negative example
    texts.append(add_noise_to_text(random.choice(negative_reviews)))
    labels.append(0)

# Convert labels to numpy array and shuffle both lists together
labels = np.array(labels)
combined = list(zip(texts, labels))
random.shuffle(combined)
texts, labels = zip(*combined)
texts = list(texts)
labels = np.array(labels)

In [3]:
# Train and evaluate classification model
print("Training document classification model...")
dataset = Dataset.from_dict({
    "text": texts,
    "label": labels
})
dataset = dataset.train_test_split(test_size=0.2, seed=42)

model = SetFitModel.from_pretrained(
    "all-MiniLM-L6-v2",
    use_differentiable_head=True,
    head_params={"out_features": len(set(labels))}, # Number of classes to predict
)

# Initialize trainer with improved training parameters
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    batch_size=16,
    num_iterations=20,  
    learning_rate=3e-5, # Large learning rate for this example, but this is not recommended for most cases
    metric="accuracy",
)

# Train the model
trainer.train()

# Make predictions
preds_classification = model.predict(texts)
acc_classification = accuracy_score(labels, preds_classification.cpu().numpy())
print(f"Document Classification Accuracy: {acc_classification:.2f}")

# Test sentiment
new_texts = [
    "This was above and beyond my expectations",
    "I regret this purchase"
]
preds = model.predict(new_texts)
for review, pred in zip(new_texts, preds):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Text: '{review}' -> Sentiment: {sentiment}")

Training document classification model...


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/var/folders/6y/y7jr3zbd39b6fvmxt9lwswl00000gn/T/ipykernel_47035/1238241322.py:16: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 3200
  Batch size = 16
  Num epochs = 1


  0%|          | 0/200 [00:00<?, ?it/s]

...

The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 256.


{'train_runtime': 25.2003, 'train_samples_per_second': 126.983, 'train_steps_per_second': 7.936, 'train_loss': 0.026974876895546913, 'epoch': 1.0}


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Document Classification Accuracy: 1.00
Text: 'This was above and beyond my expectations' -> Sentiment: Positive
Text: 'I regret this purchase' -> Sentiment: Negative
